In [ ]:
import pandas as pd
from sqlalchemy.orm import sessionmaker
from src.db.database import connect
from src.config.states import *
from src.db.database import Repository

In [ ]:
with connect() as session:
    repositories = pd.read_sql_table("repositories", session.connection())
    total = len(repositories)
    print("A total of {} repositories were retrieved ".format(total))

In [ ]:
display(repositories.state.value_counts())

In [ ]:
selected = repositories[repositories.state==REP_SELECTED].copy()
selected["disk_usage"] = selected["disk_usage"].astype(int)

print(
    "Disk Usage for the {} repositories is estimated to be:\n"
    "\033[92m{} KB - {:.2f} MB - {:.2f} GB - {:.2f} TB\033[0m\n"
    .format(
        len(selected),
        selected.disk_usage.sum(),
        selected.disk_usage.sum() / 10 ** 3,
        selected.disk_usage.sum() / 10 ** 6,
        selected.disk_usage.sum() / 10 ** 9
    )
)

partitions = 3
print("Spliting in {} partitions, each will process {:.2f} GB".format(partitions, selected.disk_usage.sum()/ 10 ** 6 / 3))

In [ ]:
selected['cumulative_sum'] = selected['disk_usage'].cumsum()
partition_size = selected.disk_usage.sum() / 3
split_points = [partition_size, partition_size * 2]

selected_part1 = selected[selected['cumulative_sum'] <= split_points[0]].copy()
selected_part2 = selected[(selected['cumulative_sum'] > split_points[0]) & (selected['cumulative_sum'] <= split_points[1])].copy()
selected_part3 = selected[selected['cumulative_sum'] > split_points[1]].copy()
display(selected_part1, selected_part2, selected_part3)

In [ ]:
import os
from src.db.database import Base
from src.config.consts import DB_DIR
from sqlalchemy import create_engine
engine1 = create_engine("sqlite:////{}part1_dsmining.sqlite".format(DB_DIR + os.sep), convert_unicode=True, echo=False)
engine2 = create_engine("sqlite:////{}part2_dsmining.sqlite".format(DB_DIR + os.sep), convert_unicode=True, echo=False)
engine3 = create_engine("sqlite:////{}part3_dsmining.sqlite".format(DB_DIR + os.sep), convert_unicode=True, echo=False)
Base.metadata.create_all(engine1)
Base.metadata.create_all(engine2)
Base.metadata.create_all(engine3);

In [ ]:
part1_data = selected_part1.to_dict(orient='records')
Session1 = sessionmaker(bind=engine1)
session1 = Session1()
session1.execute("INSERT INTO sqlite_sequence (name, seq) VALUES "
                 "('cell_data_ios', 100000000000),"
                 "('cell_markdown_features', 100000000000),"
                 "('cell_modules', 100000000000),"
                 "('cells', 100000000000),"
                 "('commits', 100000000000),"
                 "('data_ios', 100000000000),"
                 "('extractions', 100000000000),"
                 "('modules', 100000000000),"
                 "('notebook_markdowns', 100000000000),"
                 "('notebooks', 100000000000),"
                 "('python_file_data_ios', 100000000000),"
                 "('python_file_modules', 100000000000),"
                 "('python_files', 100000000000),"
                 "('requirement_files', 100000000000);")
session1.bulk_insert_mappings(Repository, part1_data)
session1.commit()
session1.close()

In [ ]:
part2_data = selected_part2.to_dict(orient='records')
Session2 = sessionmaker(bind=engine2)
session2 = Session2()
session2.execute("INSERT INTO sqlite_sequence (name, seq) VALUES "
                 "('cell_data_ios', 200000000000),"
                 "('cell_markdown_features', 200000000000),"
                 "('cell_modules', 200000000000),"
                 "('cells', 200000000000),"
                 "('commits', 200000000000),"
                 "('data_ios', 200000000000),"
                 "('extractions', 200000000000),"
                 "('modules', 200000000000),"
                 "('notebook_markdowns', 200000000000),"
                 "('notebooks', 200000000000),"
                 "('python_file_data_ios', 200000000000),"
                 "('python_file_modules', 200000000000),"
                 "('python_files', 200000000000),"
                 "('requirement_files', 200000000000);")
session2.bulk_insert_mappings(Repository, part2_data)
session2.commit()

In [ ]:
selected_part3.loc[selected_part3['id'] == 326419, 'git_pushed_at'] = '2023-03-27 11:06:07'
selected_part3.loc[selected_part3['id'] == 326458, 'git_pushed_at'] = '2023-03-26 23:11:42'
selected_part3.loc[selected_part3['id'] == 326481, 'git_pushed_at'] = '2023-03-27 00:43:24'
part3_data = selected_part3.to_dict(orient='records')
Session3 = sessionmaker(bind=engine3)
session3 = Session3()
session3.execute("INSERT INTO sqlite_sequence (name, seq) VALUES "
                 "('cell_data_ios', 30000000000),"
                 "('cell_markdown_features', 30000000000),"
                 "('cell_modules', 30000000000),"
                 "('cells', 30000000000),"
                 "('commits', 30000000000),"
                 "('data_ios', 30000000000),"
                 "('extractions', 30000000000),"
                 "('modules', 30000000000),"
                 "('notebook_markdowns', 30000000000),"
                 "('notebooks', 30000000000),"
                 "('python_file_data_ios', 30000000000),"
                 "('python_file_modules', 30000000000),"
                 "('python_files', 30000000000),"
                 "('requirement_files', 30000000000);")
session3.bulk_insert_mappings(Repository, part3_data)
session3.commit()